In [1]:
"""Make a quantum circuit whose device is the
22 qubit Foxtail quantum computer.
"""
import cirq
import numpy as np
from math import pi
#from cirq.ops import gate_operation
from cirq.value import Duration

#libraries to plot histogram
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

#google
from cirq import circuits, ops, sim, study, protocols, optimizers
from cirq.google import convert_to_xmon_gates
from cirq import circuits, ops, sim, study, protocols
from cirq.google import xmon_device

from cirq.devices import GridQubit

shots=1000
# grab the device
foxtail = cirq.google.Foxtail

# get the qubits of foxtail
qubits = sorted(list(foxtail.qubits))

# make a circuit and set foxtail to be its device
circ = cirq.Circuit(device=foxtail)

N=2

In [ ]:
"""Print out the architecture of the FoxTail quantum computer."""
s = "FoxTail has {} qubits arranged as follows:\n"
print(s.format(len(cirq.google.Foxtail.qubits)))
print(cirq.google.Foxtail)
#implement a Controlled-Z gate



In [13]:
"""Compile/decompose operations into the native gate set of Foxtail."""
options = cirq.google.sim.XmonOptions(num_shards=2)
simulator = cirq.google.XmonSimulator(options)

# get the hadamard gate
hgate = cirq.H(qubits[0])   
cx_0_1 = cirq.CNOT(qubits[0],qubits[1])
cx_1_2 = cirq.CNOT(qubits[1],qubits[2])

sym = cirq.Symbol("j")
phi = (pi*sym)/(N+1)
matrix=np.array([[1, 0], [0, np.exp(1.0j*phi)]])

cirq.single_qubit_matrix_to_pauli_rotations(matrix)

# compile/decompose the gate into native operations
compiled_hgate = foxtail.decompose_operation(hgate)
compiled_cx_0_1gate = foxtail.decompose_operation(cx_0_1)
compiled_cx_1_2gate = foxtail.decompose_operation(cx_1_2)

# print out the gate and its compilation
print(hgate, "=", *compiled_hgate)
print(cx_0_1, "=", *compiled_cx_0_1gate)
print(cx_1_2, "=", *compiled_cx_1_2gate)

circ.append(hgate)
circ.append(cx_0_1)
circ.append(cx_1_2)

# print the circuit
print(circ)

AttributeError: module 'cirq' has no attribute 'Symbol'

In [3]:
res = simulator.simulate(circ)
print(res.dirac_notation())

-0.71j|000⟩ - 0.71j|111⟩


In [4]:
options = cirq.google.sim.XmonOptions(num_shards=2)
simulator = cirq.google.XmonSimulator(options)

# get the hadamard gate
hgate = cirq.H(qubits[0])   
cx_0_1 = cirq.CNOT(qubits[0],qubits[1])
cx_1_2 = cirq.CNOT(qubits[1],qubits[2])

circ.append(hgate)
circ.append(cx_0_1)
circ.append(cx_1_2)

# print the circuit
print(circ)

res = simulator.simulate(circ)
print(res.dirac_notation())

(0, 0): ───Y^-0.5────Z────────Z^0.75───X^0.5───S^-1───Y^-0.5───@───S^-1───Y^0.5───Y^0.5─────Z^-0.75───Y^-0.5───Z──────Z^0.75───X^0.5───S^-1───Y^-0.5──────────────────────────────────────────@───S^-1───Y^0.5───Y^0.5─────Z^-0.75──────────────────────────────────────────────────────────────────
                                                               │                                                                                                                              │
(0, 1): ───X^-0.25───Y^-0.5────────────────────────────────────@───S^-1───Y^0.5───X^-0.25───Z^0.75────X^0.5────S^-1───Y^-0.5───@───────S^-1───Y^0.5────Y^0.5─────Z^-0.75───X^-0.25───Y^-0.5───@───S^-1───Y^0.5───X^-0.25───Z^0.75────X^0.5───S^-1───Y^-0.5───@───S^-1───Y^0.5───Y^0.5─────Z^-0.75───
                                                                                                                               │                                                                                              

In [ ]:
"""Attempt to optimize the circuit. Only commutes
Z across CZ for a single Bell state preparation circuit, 
simplifies two sequential Bell state preparation circuits. 
(Run the cell above twice to see this.)
"""
ejectPaulis = cirq.optimizers.EjectPhasedPaulis()
ejectZ = cirq.optimizers.EjectZ()
ejectZ.optimize_circuit(circ)

ejectPaulis.optimize_circuit(circ)
drop_neg = cirq.optimizers.DropNegligible()
drop_neg.optimize_circuit(circ)
cirq.merge_single_qubit_gates_into_phased_x_z(circ)

print(circ)

res = simulator.simulate(circ)
print(res.dirac_notation())

In [8]:
"""See how long each native operation takes to implement."""
# grab the times of some native gates
ytime = foxtail.duration_of(cirq.Y(qubits[0]))
ztime = foxtail.duration_of(cirq.Z(qubits[0]))
cztime = foxtail.duration_of(cirq.CZ(qubits[0], qubits[1]))

# print them out
print("Y time =", ytime.total_nanos(), "nanoseconds.")
print("Z time =", ztime.total_nanos(), "nanoseconds.")
print("CZ time =", cztime.total_nanos(), "nanoseconds.")

Y time = 20.0 nanoseconds.
Z time = 0.0 nanoseconds.
CZ time = 50.0 nanoseconds.


In [ ]:
ejectPaulis.optimize_circuit(circ)
print(circ)